# Assignment 2

## Kaggle Competition

## Santander Customer Transaction Prediction
At Santander our mission is to help people and businesses prosper. We are always looking for ways to help our customers understand their financial health and identify which products and services might help them achieve their monetary goals.
Our data science team is continually challenging our machine learning algorithms, working with the global data science community to make sure we can more accurately identify new ways to solve our most common challenge, binary classification problems such as: is a customer satisfied? Will a customer buy this product? Can a customer pay this loan?
In this challenge, we invite Kagglers to help us identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data we have available to solve this problem.
## Dataset
Tha dataset is taken from https://www.kaggle.com/c/santander-customer-transaction-prediction/data
File descriptions
train.csv - the training set.
test.csv - the test set.
The test set contains some rows which are not included in scoring. sample_submission.csv - a sample submission file in the correct format.
## Abstract
At Santander our mission is to help people and help businesses prosper. We are always looking for ways to help our customers understand their financial health and identify which products and services might help them achieve their monetary goals.
Our method is firstly to analyze the distribution of the dataset, the condition of balance in the dataset, average number and standard deviation of both rows and columns.
Then we use the H2O AUTOML platform to generate the best models, and use the best model to predict the value in the test datset, and the rank is 1948 out of 3000 which was not a good score and then we started to train the model using LightGBM and predicted the target. The final step is tunning the hyperparameters of the model by adding more number of leaves, maxbin,reducing the learning rate and adding more parameters. We got even better outcome of 589 out of 3000 after tunning the hyperparameters and training the best model to predict the results.

## Import Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
sns.set_style('whitegrid')
import time
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("C:/Users/87421/Desktop/Kaggle/santander"))


['.ipynb_checkpoints', 'sample_submission.csv', 'Santander_3.ipynb', 'STCP_lightgbm.ipynb', 'submission.csv', 'test.csv', 'train.csv']


## Import the training dataset

In [2]:
train_df = pd.read_csv('C:/Users/87421/Desktop/Kaggle/santander/train.csv')
print('Rows: ',train_df.shape[0],'Columns: ',train_df.shape[1])
train_df.info()

Rows:  200000 Columns:  202
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [3]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
train_df['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

## Import the test dataset

In [5]:
test_df = pd.read_csv('C:/Users/87421/D
                      esktop/Kaggle/santander/test.csv')

In [6]:
X_test = test_df.drop('ID_code',axis=1)

In [7]:
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [8]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

## Create lightGBM model

In [9]:
params = {'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [11]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
            
    #y_pred_valid = model.predict(X_valid)
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/5


Fold 0 started at Thu Feb 28 22:51:04 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.827918	valid_1's auc: 0.814605
[600]	training's auc: 0.862802	valid_1's auc: 0.845745
[900]	training's auc: 0.880631	valid_1's auc: 0.860844
[1200]	training's auc: 0.891953	valid_1's auc: 0.869908
[1500]	training's auc: 0.900115	valid_1's auc: 0.876263
[1800]	training's auc: 0.906338	valid_1's auc: 0.88068
[2100]	training's auc: 0.911263	valid_1's auc: 0.884034
[2400]	training's auc: 0.915172	valid_1's auc: 0.886626
[2700]	training's auc: 0.918385	valid_1's auc: 0.888726
[3000]	training's auc: 0.921196	valid_1's auc: 0.890411
[3300]	training's auc: 0.923598	valid_1's auc: 0.891873
[3600]	training's auc: 0.925717	valid_1's auc: 0.893051
[3900]	training's auc: 0.927648	valid_1's auc: 0.894246
[4200]	training's auc: 0.929203	valid_1's auc: 0.895105
[4500]	training's auc: 0.930731	valid_1's auc: 0.895752
[4800]	training's auc: 0.932079	valid_1's auc: 0.896317
[5

### The LightGBM model gave the accuracy of 89.48% using the Santander Customer Transcation Prediction

In [106]:

"""from catboost import CatBoostClassifier,Pool
prediction1 = np.zeros(len(X_test))
m = CatBoostClassifier(loss_function="Logloss",eval_metric="AUC",
                       boosting_type = 'Ordered')
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    train_data = Pool(X_train, label=y_train)
    valid_data = Pool(X_valid, label=y_valid)

    model1 = m.fit(train_data,eval_set=valid_data,use_best_model=True,verbose=300)
    
    prediction1 += model1.predict(X_test)/5
"""

'from catboost import CatBoostClassifier,Pool\nprediction1 = np.zeros(len(X_test))\nm = CatBoostClassifier(loss_function="Logloss",eval_metric="AUC",\n                       boosting_type = \'Ordered\')\nfor fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):\n    print(\'Fold\', fold_n, \'started at\', time.ctime())\n    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]\n    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]\n\n    train_data = Pool(X_train, label=y_train)\n    valid_data = Pool(X_valid, label=y_valid)\n\n    model1 = m.fit(train_data,eval_set=valid_data,use_best_model=True,verbose=300)\n    \n    prediction1 += model1.predict(X_test)/5\n'

## Create xGBoost model

In [90]:
mod = xgb.XGBClassifier(max_depth=4,n_estimators=999999, colsample_bytree=0.7,subsample = 0.7, 
                              min_child_weight = 50, eval_metric = "auc",gamma = 5,alpha = 0,
                               booster = "gbtree",colsample_bylevel = 0.7, learning_rate=0.1,
                              objective='binary:logistic', n_jobs=-1)

prediction2 = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    #evallist = [(valid_data, 'eval'), (train_data, 'train')]
    model2 = mod.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],verbose=200, eval_metric='auc',
                        early_stopping_rounds=200)
    
    prediction2 += model2.predict(X_test, ntree_limit=model2.best_ntree_limit)/5

Fold 0 started at Fri Mar  1 16:18:40 2019
[0]	validation_0-auc:0.60766
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.871004
[400]	validation_0-auc:0.886465
[600]	validation_0-auc:0.891015
[800]	validation_0-auc:0.892374
[1000]	validation_0-auc:0.892688
Stopping. Best iteration:
[959]	validation_0-auc:0.892841

Fold 1 started at Fri Mar  1 16:32:03 2019
[0]	validation_0-auc:0.598816
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.872368
[400]	validation_0-auc:0.887945
[600]	validation_0-auc:0.893176
[800]	validation_0-auc:0.894424
[1000]	validation_0-auc:0.894634
[1200]	validation_0-auc:0.894634
Stopping. Best iteration:
[1176]	validation_0-auc:0.894919

Fold 2 started at Fri Mar  1 16:47:28 2019
[0]	validation_0-auc:0.575435
Will train until validation_0-auc hasn't improved in 200 rounds.
[200]	validation_0-auc:0.875978
[400]	validation_0-auc:0.891846
[600]	validation_0-auc:0.897075
[800]	validation_

### The XGBoost model gave the accuracy of 89.21% using the Santander Customer Transcation Prediction

<table style="text-align:left"> 
    <thead> 
    <tr> <th>Number</th> <th>Model Name</th> <th>Accuray</th> </tr> </thead> <tbody> 
    <tr> 
        <tr> <th scope='row'>2</th> <td style="text-align:center">XGBoosting</td>  <td>0.8921</td> </tr>
        <tr> <th scope='row'>3</th> <td style="text-align:center">CatBoosting</td>  <td>0.8910</td> </tr>
    <tr> <th scope='row'>4</th> <td style="text-align:center">LightGBM</td>  <td>0.8948</td> </tr>


In [91]:
prediction

array([0.08114897, 0.2424331 , 0.21381503, ..., 0.003254  , 0.09354758,
       0.07144961])

In [92]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)

In [93]:
#sub["target"] = prediction1
#sub.to_csv("submission1.csv", index=False)

In [94]:
#sub["target"] = (prediction + prediction1)/2
#sub.to_csv("submission2.csv", index=False)

In [95]:
#sub["target"] = prediction2
#sub.to_csv("submission3.csv", index=False)

In [96]:
#sub["target"] = (prediction + prediction2)/2 
#sub.to_csv("submission4.csv", index=False)

In [97]:
#sub["target"] = (prediction + prediction1 + prediction2)/3 
#sub.to_csv("submission5.csv", index=False)

## Tune the hyperparameters of lightGBM

LightGBM uses the leaf-wise tree growth algorithm, while many other popular tools use depth-wise tree growth. Compared with depth-wise growth, the leaf-wise algorithm can converge much faster. However, the leaf-wise growth may be over-fitting if not used with the appropriate parameters

For Faster Speed
Use bagging by setting bagging_fraction and bagging_freq
Use feature sub-sampling by setting feature_fraction
Use small max_bin
Use save_binary to speed up data loading in future learning

For Better Accuracy
Use large max_bin (may be slower)
Use small learning_rate with large num_iterations
Use large num_leaves (may cause over-fitting)
Use bigger training data

Create the variables used and define the range  

In [72]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier()
params = {'num_leaves': [8, 10, 12],  #Enlarge the range of num_leaves
         'min_data_in_leaf': [42],
         'objective': ['binary'],
         'max_depth': [14, 16, 18],   #Enlarge the range of max_depth(tree based hyperparameters)
         'learning_rate': [0.0123],
         'boosting': ['gbdt'],
         'bagging_freq': [5],
         'bagging_fraction': [0.8],
         'feature_fraction': [0.8201],
         'bagging_seed': [11],
         'reg_alpha': [1.728910519108444],
         'reg_lambda': [4.9847051755586085],
         'random_state': [42],
         'metric': ['auc'],
         'verbosity': [-1],
         'subsample': [0.81],
         'min_gain_to_split': [0.01077313523861969],
         'min_child_weight': [19.428902804238373],
         'num_threads': [4, 8]}

Create the grid search object

In [73]:
grid_search = GridSearchCV(estimator=lgbm, param_grid=params, cv = 5, n_jobs=-1, verbose=2)

In [74]:
X_train = train_df.drop(columns=['target', 'ID_code'], inplace=False)
y_train = train_df.target

Train the grid search model

In [75]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.7min
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [8, 10, 12], 'min_data_in_leaf': [42], 'objective': ['binary'], 'max_depth': [14, 16, 18], 'learning_rate': [0.0123], 'boosting': ['gbdt'], 'bagging_freq': [5], 'bagging_fraction': [0.8], 'feature_fraction': [0.8201], 'bagging_seed': [11], 'reg_alpha': [1.728910519108444], ...n_to_split': [0.01077313523861969], 'min_child_weight': [19.428902804238373], 'num_threads': [4, 8]},
       pre_dispatch='2*n_jobs', refit=True, ret

The hyperparameters we want to tune is num_leave, max_depth, and the number of threads

In [98]:
grid_search.best_params_

{'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'bagging_seed': 11,
 'boosting': 'gbdt',
 'feature_fraction': 0.8201,
 'learning_rate': 0.0123,
 'max_depth': 10,
 'metric': 'auc',
 'min_child_weight': 19.428902804238373,
 'min_data_in_leaf': 42,
 'min_gain_to_split': 0.01077313523861969,
 'num_leaves': 4,
 'num_threads': 4,
 'objective': 'binary',
 'random_state': 42,
 'reg_alpha': 1.728910519108444,
 'reg_lambda': 4.9847051755586085,
 'subsample': 0.81,
 'verbosity': -1}

### As we can see, num_leaves = 8 from [8, 10, 12], num_threads = 4 from [4, 8], max_depth = 14 from [14, 16, 18]
### So we can't let them become bigger. We have to reduce it.

In [77]:
grid_search.best_score_

0.89951

the score is pretty good

#### Lets define the lower range of them.

In [78]:
params = {'num_leaves': [4, 6, 8],
         'min_data_in_leaf': [42],
         'objective': ['binary'],
         'max_depth': [10, 12, 14],
         'learning_rate': [0.0123],
         'boosting': ['gbdt'],
         'bagging_freq': [5],
         'bagging_fraction': [0.8],
         'feature_fraction': [0.8201],
         'bagging_seed': [11],
         'reg_alpha': [1.728910519108444],
         'reg_lambda': [4.9847051755586085],
         'random_state': [42],
         'metric': ['auc'],
         'verbosity': [-1],
         'subsample': [0.81],
         'min_gain_to_split': [0.01077313523861969],
         'min_child_weight': [19.428902804238373],
         'num_threads': [4]}

In [81]:
grid_search = GridSearchCV(estimator=grid_search.best_estimator_, param_grid=params, cv = 5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\87421\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\Users\87421\Anaconda3\lib\s

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMClassifier(bagging_fraction=0.55, bagging_freq=5, bagging_seed=31415,
        boost_from_average=False, boosting='gbdt', boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, data_random_seed=31415,
        drop_seed=31415, feature_fraction=0.51,
        feature_fracti...subsample=0.81,
        subsample_for_bin=200000, subsample_freq=0, verbose=1,
        verbosity=-1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [4, 6, 8], 'min_data_in_leaf': [42], 'objective': ['binary'], 'max_depth': [10, 12, 14], 'learning_rate': [0.0123], 'boosting': ['gbdt'], 'bagging_freq': [5], 'bagging_fraction': [0.8], 'feature_fraction': [0.8201], 'bagging_seed': [11], 'reg_alpha': [1.728910519108444], 'r...gain_to_split': [0.01077313523861969], 'min_child_weight': [19.428902804238373], 'num_threads': [4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
      

#### the score becomes much lower, so we can see that num_leaves = 8 , num_threads = 4 , max_depth = 14 are the good set of hyperparamters to choose

In [82]:
grid_search.best_score_

0.673515

In [50]:
grid_search.best_estimator_

LGBMClassifier(bagging_fraction=0.55, bagging_freq=5, bagging_seed=31415,
        boost_from_average=False, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, data_random_seed=31415, drop_seed=31415,
        feature_fraction=0.51, feature_fraction_seed=31415,
        importance_type='split', is_unbalance=True, learning_rate=0.0005,
        max_bin=94, max_depth=14, metric='auc', min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=45, min_split_gain=0.0,
        min_sum_hessian_in_leaf=0.000446, n_estimators=100, n_jobs=-1,
        num_leaves=14, objective='binary', random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, save_binary=True, seed=31452,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0, verbose=1)

So, we should use the parameters before.

In [83]:
params = {'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'bagging_seed': 11,
          'boosting': 'gbdt',
          'feature_fraction': 0.8201,
          'learning_rate': 0.0123,
          'max_depth': 14,
          'metric': 'auc',
          'min_child_weight': 19.428902804238373,
          'min_data_in_leaf': 42,
          'min_gain_to_split': 0.01077313523861969,
          'num_leaves': 8,
          'num_threads': 4,
          'objective': 'binary',
          'random_state': 42,
          'reg_alpha': 1.728910519108444,
          'reg_lambda': 4.9847051755586085,
          'subsample': 0.81,
          'verbosity': -1}

## Run the lightGBM model again.

In [84]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
            
    #y_pred_valid = model.predict(X_valid)
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/5

Fold 0 started at Fri Mar  1 01:01:16 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.827918	valid_1's auc: 0.814605
[600]	training's auc: 0.862802	valid_1's auc: 0.845745
[900]	training's auc: 0.880631	valid_1's auc: 0.860844
[1200]	training's auc: 0.891953	valid_1's auc: 0.869908
[1500]	training's auc: 0.900115	valid_1's auc: 0.876263
[1800]	training's auc: 0.906338	valid_1's auc: 0.88068
[2100]	training's auc: 0.911263	valid_1's auc: 0.884034
[2400]	training's auc: 0.915172	valid_1's auc: 0.886626
[2700]	training's auc: 0.918385	valid_1's auc: 0.888726
[3000]	training's auc: 0.921196	valid_1's auc: 0.890411
[3300]	training's auc: 0.923598	valid_1's auc: 0.891873
[3600]	training's auc: 0.925717	valid_1's auc: 0.893051
[3900]	training's auc: 0.927648	valid_1's auc: 0.894246
[4200]	training's auc: 0.929203	valid_1's auc: 0.895105
[4500]	training's auc: 0.930731	valid_1's auc: 0.895752
[4800]	training's auc: 0.932079	valid_1's auc: 0.896317
[5

In [85]:
prediction

array([0.08114897, 0.2424331 , 0.21381503, ..., 0.003254  , 0.09354758,
       0.07144961])

In [87]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission_6.csv", index=False)

# Liscense
Copyright (c) 2019 Hao Zhou

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Citations
Exploratory Data Analysis: https://www.kaggle.com/gpreda/santander-eda-and-prediction

LightGBM : https://www.kaggle.com/deepak525/sctp-lightgbm-lb-0-899

XGBoost Classifier : https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn

XGBoost Regression : https://machinelearningmastery.com/tune-number-size-decision-trees-xgboost-python/

Hyper parameter tuning: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

H2O AutoML: https://github.com/nikbearbrown/Kaggle/blob/master/NBB_Zillow_House_Price_Prediction_Feature_Selection.ipynb

# Conclusion
We used the Santander_Transaction_Customer_Prediction dataset to predict the target by initially running the base model on H2O AutoML and got ensembled stack learning as the best model with an accuracy of 87.23%. As the first part of the assignment we got a rank of 1948 out of 3000 in the Kaggle Competition which was not a good score and then we started to train the model using LightGBM and predicted the target.We ran 3 models which were LightGBM, catboost and XGboost. Lightgbm predicted the best accuracy out of all the 3 models. The final step was tunning the hyperparameters of the model by adding more number of leaves, maxbin,reducing the learning rate,max_depth and num_leaves.We got an accuracy of 89.95% which was better than previous model accuracy.We got even better outcome of 589 out of 3000 in the Kaggle Cometition leaderboard after tunning the hyperparameters.Finally, we conclude that out of all the models, lightgbm performed well which gave the accuracy of 89.48% and after hyperparameter tuning the accuracy changed to 89.95%.

<table style="text-align:left"> 
    <thead> 
    <tr> <th>Number</th> <th>Model Name</th> <th>Accuray</th> </tr> </thead> <tbody> 
    <tr> 
         <th scope='row'>1</th> <td style="text-align:center">H2O AutoML</td> <td>0.8723</td> </tr> 
        <tr> <th scope='row'>2</th> <td style="text-align:center">XGBoosting</td>  <td>0.8921</td> </tr>
        <tr> <th scope='row'>3</th> <td style="text-align:center">CatBoosting</td>  <td>0.8910</td> </tr>
    <tr> <th scope='row'>4</th> <td style="text-align:center">LightGBM</td>  <td>0.8948</td> </tr>
            <tr> <th scope='row'>5</th> <td style="text-align:center">Hyperparameter Tuning</td>  <td>0.8995</td> </tr>


# Contribution
1)The Santander_Transaction_Customer_Prediction dataset which we used we had to predict the probability to identify which products and services might help them achieve their monetary goals.

2)The first task was performing H2O AutoML which gave the best model for our predction. We referred Professor Nik Brown's notebook for that and we reduced the features from 201 to 44. Along with that, we also ranked the importance of the variables and deleted some variables according to the threshold and then used that dataset to be trained by H2O.

3)We extended the model by hyperparameter tunning using GridSearch CV to improve the model performance.In this we tunned the hyperparameters of the model by adding more number of leaves, maxbin,reducing the learning rate,max_depth and num_leaves.

4)We made use of Boosting ensembled learning methods and we used an additional boosting which was lightGBM and Catboosting.